# Neural Machine Translation with Attention

In [12]:
from keras.layers import Bidirectional, LSTM, Dense, Activation
from keras.layers import Concatenate, Permute, Dot, Input, RepeatVector, Lambda,Multiply
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model,Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt

%matplotlib inline 

## 1. Dataset

In [13]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 22124.09it/s]


In [17]:
dataset[0:10]

[('9 may 1998', '1998-05-09'),
 ('10.09.70', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('08 sep 1999', '1999-09-08'),
 ('1 jan 1981', '1981-01-01'),
 ('monday may 22 1995', '1995-05-22')]

In [27]:
Tx = 30
Ty = 10

X, Y, Xoh, Yoh = preprocess_data(dataset,human_vocab,machine_vocab,Tx, Ty)

print("X.shape = ", X.shape)
print("Y.shape = ", Y.shape)
print("Xoh.shape = ", Xoh.shape)
print("Yoh.shape = ", Yoh.shape)

X.shape =  (10000, 30)
Y.shape =  (10000, 10)
Xoh.shape =  (10000, 30, 37)
Yoh.shape =  (10000, 10, 11)


In [39]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2. Neural machine tranlation with attention

In [42]:
repeator = RepeatVector(Tx)
concatenate = Concatenate(axis=-1)
densor = Dense(1,activation="relu")
activator = Activation(softmax, name = "attention_weights")
dotor = Dot(axes = 1)

In [43]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenate([a, s_prev])
    e = densor(concat)
    alphas = activator(e)
    
    context = dotor([alphas, a])
    
    return context

In [44]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state=True)
output_layer = Dense(len(machine_vocab),activation=softmax)

In [51]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name = "s0")
    c0 = Input(shape=(n_s,), name = "c0")
    s = s0
    c = c0
    outputs = []
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    model = Model([X, s0, c0], outputs)
    return model

In [52]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [53]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 30, 128)      52224       input_4[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 30, 128)      0           s0[0][0]                         
                                                                 lstm_1[10][0]                    
          

                                                                 bidirectional_3[0][0]            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 131584      dot_2[10][0]                     
                                                                 s0[0][0]                         
                                                                 c0[0][0]                         
                                                                 dot_2[11][0]                     
                                                                 lstm_1[10][0]                    
                                                                 lstm_1[10][2]                    
          

In [54]:
opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=['accuracy'])

In [55]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [56]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

Epoch 1/1


 2800/10000 [=======>......................] - ETA: 10:11 - loss: 23.9189 - dense_3_loss: 2.3978 - dense_3_acc: 0.1100 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0500 - dense_3_acc_4: 0.3400 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0800 - dense_3_acc_7: 0.2200 - dense_3_acc_8: 0.1400 - dense_3_acc_9: 0.10 - ETA: 5:11 - loss: 23.5449 - dense_3_loss: 2.4328 - dense_3_acc: 0.0550 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0250 - dense_3_acc_4: 0.6700 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0400 - dense_3_acc_7: 0.6100 - dense_3_acc_8: 0.0700 - dense_3_acc_9: 0.0500 - ETA: 3:30 - loss: 23.0676 - dense_3_loss: 2.5408 - dense_3_acc: 0.0367 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0167 - dense_3_acc_4: 0.7800 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0267 - dense_3_acc_7: 0.7400 - dense_3_acc_8: 0.0467 - dense_3_acc_9: 0.033 - ETA: 2:41 - loss: 23.0706 - dense_3_loss: 2.8570 - d

 5700/10000 [================>.............] - ETA: 29s - loss: 20.8849 - dense_3_loss: 2.7660 - dense_3_acc: 0.0283 - dense_3_acc_1: 0.4224 - dense_3_acc_2: 0.1638 - dense_3_acc_3: 0.0203 - dense_3_acc_4: 0.9417 - dense_3_acc_5: 3.4483e-04 - dense_3_acc_6: 0.0028 - dense_3_acc_7: 0.9334 - dense_3_acc_8: 0.0959 - dense_3_acc_9: 0.05 - ETA: 28s - loss: 20.8212 - dense_3_loss: 2.7688 - dense_3_acc: 0.0480 - dense_3_acc_1: 0.4290 - dense_3_acc_2: 0.1680 - dense_3_acc_3: 0.0210 - dense_3_acc_4: 0.9437 - dense_3_acc_5: 3.3333e-04 - dense_3_acc_6: 0.0027 - dense_3_acc_7: 0.9147 - dense_3_acc_8: 0.1023 - dense_3_acc_9: 0.05 - ETA: 27s - loss: 20.7475 - dense_3_loss: 2.7695 - dense_3_acc: 0.0671 - dense_3_acc_1: 0.4358 - dense_3_acc_2: 0.1742 - dense_3_acc_3: 0.0229 - dense_3_acc_4: 0.9455 - dense_3_acc_5: 3.2258e-04 - dense_3_acc_6: 0.0026 - dense_3_acc_7: 0.9174 - dense_3_acc_8: 0.1061 - dense_3_acc_9: 0.05 - ETA: 26s - loss: 20.6812 - dense_3_loss: 2.7705 - dense_3_acc: 0.0831 - dense_3_acc

 8600/10000 [========================>.....] - ETA: 12s - loss: 18.5004 - dense_3_loss: 2.6986 - dense_3_acc: 0.3560 - dense_3_acc_1: 0.5922 - dense_3_acc_2: 0.2581 - dense_3_acc_3: 0.0400 - dense_3_acc_4: 0.9524 - dense_3_acc_5: 0.1943 - dense_3_acc_6: 0.0219 - dense_3_acc_7: 0.8978 - dense_3_acc_8: 0.2134 - dense_3_acc_9: 0.08 - ETA: 12s - loss: 18.4078 - dense_3_loss: 2.6929 - dense_3_acc: 0.3653 - dense_3_acc_1: 0.5969 - dense_3_acc_2: 0.2600 - dense_3_acc_3: 0.0398 - dense_3_acc_4: 0.9532 - dense_3_acc_5: 0.2032 - dense_3_acc_6: 0.0232 - dense_3_acc_7: 0.8995 - dense_3_acc_8: 0.2159 - dense_3_acc_9: 0.08 - ETA: 12s - loss: 18.3100 - dense_3_loss: 2.6869 - dense_3_acc: 0.3737 - dense_3_acc_1: 0.6027 - dense_3_acc_2: 0.2622 - dense_3_acc_3: 0.0420 - dense_3_acc_4: 0.9540 - dense_3_acc_5: 0.2137 - dense_3_acc_6: 0.0237 - dense_3_acc_7: 0.9012 - dense_3_acc_8: 0.2177 - dense_3_acc_9: 0.09 - ETA: 11s - loss: 18.2490 - dense_3_loss: 2.6843 - dense_3_acc: 0.3795 - dense_3_acc_1: 0.6064 -

10000/10000 [==============================] - ETA: 3s - loss: 16.1821 - dense_3_loss: 2.5752 - dense_3_acc: 0.5074 - dense_3_acc_1: 0.6854 - dense_3_acc_2: 0.2971 - dense_3_acc_3: 0.0666 - dense_3_acc_4: 0.9683 - dense_3_acc_5: 0.3831 - dense_3_acc_6: 0.0738 - dense_3_acc_7: 0.9318 - dense_3_acc_8: 0.2647 - dense_3_acc_9: 0.102 - ETA: 3s - loss: 16.1153 - dense_3_loss: 2.5721 - dense_3_acc: 0.5123 - dense_3_acc_1: 0.6884 - dense_3_acc_2: 0.2983 - dense_3_acc_3: 0.0675 - dense_3_acc_4: 0.9686 - dense_3_acc_5: 0.3876 - dense_3_acc_6: 0.0748 - dense_3_acc_7: 0.9326 - dense_3_acc_8: 0.2661 - dense_3_acc_9: 0.102 - ETA: 3s - loss: 16.0508 - dense_3_loss: 2.5692 - dense_3_acc: 0.5166 - dense_3_acc_1: 0.6913 - dense_3_acc_2: 0.3017 - dense_3_acc_3: 0.0676 - dense_3_acc_4: 0.9690 - dense_3_acc_5: 0.3921 - dense_3_acc_6: 0.0763 - dense_3_acc_7: 0.9334 - dense_3_acc_8: 0.2675 - dense_3_acc_9: 0.103 - ETA: 2s - loss: 15.9890 - dense_3_loss: 2.5666 - dense_3_acc: 0.5212 - dense_3_acc_1: 0.6940 - 